In [1]:
import cv2
import numpy as np

from project.common.camera.distance import StereoCameraConfig
from project.common.config_class.camera_feed_cleaner import CameraConfig

def get_right_camera_config():
    camera_matrix = np.array([
        [1411.0751692531098, 0.0, 1006.3176957089871],
        [0.0, 1408.0753545859811, 502.3819024443242],
        [0.0, 0.0, 1.0]
    ])
    dist_coeffs = np.array([
        [-0.34169888754351235, 0.13460057589343846, 0.0011882429085977764,
         0.002051994220161123, -0.033985190528539]
    ])
    return CameraConfig({
        "camera-matrix": camera_matrix.tolist(),
        "dist-coeff": dist_coeffs.tolist()
    })

def get_left_camera_config():
    camera_matrix = np.array([
        [1445.3879247959794, 0.0, 923.2765760127855],
        [0.0, 1435.2627288375536, 534.2252438992628],
        [0.0, 0.0, 1.0]
    ])
    dist_coeffs = np.array([
        [-0.44241649408964584, 0.22637048962483938, -0.0025477943655799044,
         0.002173846358793312, -0.07418904329741373]
    ])
    return CameraConfig({
        "camera-matrix": camera_matrix.tolist(),
        "dist-coeff": dist_coeffs.tolist()
    })


def get_stereo_camera_config() -> StereoCameraConfig:
    K_left = np.array([
        [1445.3879247959794, 0.0, 923.2765760127855],
        [0.0, 1435.2627288375536, 534.2252438992628],
        [0.0, 0.0, 1.0]
    ])
    D_left = np.array([
        [-0.44241649408964584, 0.22637048962483938, -0.0025477943655799044,
         0.002173846358793312, -0.07418904329741373]
    ])
    K_right = np.array([
        [1411.0751692531098, 0.0, 1006.3176957089871],
        [0.0, 1408.0753545859811, 502.3819024443242],
        [0.0, 0.0, 1.0]
    ])
    D_right = np.array([
        [-0.34169888754351235, 0.13460057589343846, 0.0011882429085977764,
         0.002051994220161123, -0.033985190528539]
    ])
    R = np.array([
        [0.9554639517379155, 0.013800152374098141, -0.2947849940614648],
        [-0.001157865385737815, 0.999073625150032, 0.043018029677356504],
        [0.2951055680211993, -0.04076085529064728, 0.9545948126817264]
    ])
    T = np.array([
        [-0.07832301156350042],
        [0.00001],
        [-0.024843288838636884]
    ])

    return StereoCameraConfig(K_left, D_left, K_right, D_right, R, T)


In [2]:
import cv2
import numpy as np

from project.common.camera.distance import calculate_distance_with_disparity, cameras_to_disparity_map
from project.common.camera.transform import unfisheye_image

# Camera setup
camera_left = cv2.VideoCapture(0)
camera_right = cv2.VideoCapture(1)

# Load camera configurations
get_left_camera_config = get_left_camera_config()  # Replace with your function
right_camera_config = get_right_camera_config()  # Replace with your function
stereo_camera_config = get_stereo_camera_config()  # Replace with your function

# Initialize cursor position
cursor_position = (300, 100)  # Default point to calculate distance

# Mouse callback function
def mouse_callback(event, x, y, flags, param):
    global cursor_position
    if event == cv2.EVENT_MOUSEMOVE:  # Update cursor position on mouse movement
        cursor_position = (x, y)

# Register mouse callback for disparity window
cv2.namedWindow("disparity")
cv2.setMouseCallback("disparity", mouse_callback)

while True:
    # Capture frames
    frame_left = unfisheye_image(camera_left.read()[1], get_left_camera_config.camera_matrix, get_left_camera_config.dist_coeff)
    frame_right = unfisheye_image(camera_right.read()[1], right_camera_config.camera_matrix, right_camera_config.dist_coeff)

    if frame_left is None or frame_right is None:
        print("Error: One of the frames is None.")
        continue

    # Compute disparity map
    disparity, normalized_disparity, Q = cameras_to_disparity_map(
        left_img=frame_left,
        right_img=frame_right,
        stereo_camera_config=stereo_camera_config,
    )

    if disparity is None or disparity.size == 0:
        print("Error: Disparity map is None or empty.")
        continue

    # Calculate distance at the current cursor position
    distance = calculate_distance_with_disparity(
        disparity, Q, point=cursor_position
    )

    # Overlay distance on the disparity map
    display_disparity = disparity.copy()
    if distance is not None:
        distance_text = f"Distance: {distance:.2f} meters"
        cv2.putText(display_disparity, distance_text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.circle(display_disparity, cursor_position, 5, (255, 0, 0), -1)

    # Show frames
    cv2.imshow("Left", frame_left)
    cv2.imshow("Right", frame_right)
    cv2.imshow("disparity", display_disparity)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release resources
camera_left.release()
camera_right.release()
cv2.destroyAllWindows()

2024-11-23 23:11:41.586 Python[94841:2263098] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0


2024-11-23 23:11:42.267 Python[94841:2263098] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparity min: -1.0, max: 15.0
Disparit